In [1]:
!mkdir /kaggle/working/train
!ln -s /kaggle/input/sp-cup-train-dataset/train_fake/fake /kaggle/working/train/fake
!ln -s /kaggle/input/sp-cup-train-dataset/train_real/real /kaggle/working/train/real

!mkdir /kaggle/working/test
!ln -s /kaggle/input/sp-cup-fake-validation/valid_fake/fake /kaggle/working/test/fake
!ln -s /kaggle/input/sp-cup-fake-validation/valid_real/real /kaggle/working/test/real

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from kaggle_datasets import KaggleDatasets

# Step 1: Initialize TPU
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.TPUStrategy(tpu)
#     print("All TPU devices: ", tf.config.list_logical_devices('TPU'))
# except ValueError:
#     print("TPU not found.")

strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

# Use GPU

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Dataset 

In [4]:
import keras

In [6]:
image_size = (224, 224)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "/kaggle/working/train",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 262160 files belonging to 2 classes.
Using 209728 files for training.
Using 52432 files for validation.


In [ ]:
# # Callbacks
# callbacks = [
#     EarlyStopping(
#         monitor='val_loss',  # Monitor validation loss
#         patience=40,          # Stop if no improvement for 5 epochs
#         restore_best_weights=True  # Revert to best model after stopping
#     ),
#     ModelCheckpoint(
#         "best_model.h5",    # Save best model during training
#         monitor='val_loss', # Monitor validation loss
#         save_best_only=True # Save only the best model
#     )
# ]

In [7]:
with strategy.scope():
    model = tf.keras.Sequential([
        EfficientNetB0(include_top=False, input_shape=(224, 224, 3), pooling='avg'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

# Step 4: Train Model
epochs = 100
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    steps_per_epoch=len(train_ds) // batch_size,
    validation_steps=len(val_ds) // batch_size',
    # callbacks=callbacks  # Add callbacks here

)

# Step 5: Save Model
model.save("efficientnet_cifake.h5")

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100


2025-01-20 16:41:01.127151: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/sequential_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


12/12 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.7112 - loss: 0.5436 - val_accuracy: 0.8281 - val_loss: 0.4686
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 924ms/step - accuracy: 0.8446 - loss: 0.3972 - val_accuracy: 0.8125 - val_loss: 0.4714
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 924ms/step - accuracy: 0.8314 - loss: 0.3754 - val_accuracy: 0.8177 - val_loss: 0.7935
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 925ms/step - accuracy: 0.8438 - loss: 0.3537 - val_accuracy: 0.8646 - val_loss: 0.9306
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 925ms/step - accuracy: 0.8984 - loss: 0.2621 - val_accuracy: 0.8281 - val_loss: 1.4950
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 925ms/step - accuracy: 0.8867 - loss: 0.2823 - val_accuracy: 0.7839 - val_loss: 1.2405
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 924ms/step - accuracy: 0.8882 - loss: 0.2680 - val_accuracy: 0.8021 - val_loss: 0.7269
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 924ms/step - accuracy: 0.8927 - loss: 0.2467 - val_accuracy: 0

# On Test Dataset

In [8]:
import tensorflow as tf

# Define the same preprocessing steps
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))  # Resize for EfficientNet
    image = image / 255.0  # Normalize to [0, 1]
    return image, label

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/working/test',
    image_size=(224, 224),  # Resize for EfficientNet
    batch_size=32,
    shuffle=False  # Do not shuffle for consistent evaluation
)

# Apply preprocessing
test_dataset = test_dataset.map(preprocess_image)


Found 3072 files belonging to 2 classes.


In [9]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)

96/96 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - accuracy: 0.8377 - loss: 0.6635
